# MACHINE LEARNING PARA CALCULAR A EVASÃO DOS ALUNOS DO PERÍODO EAD

### USANDO RANDOM FOREST

### IMPORTANDO TUDO QUE VAI SER USADO NA ML

In [ ]:
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import ADASYN
from collections import Counter
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pickle


### IMPORTANDO A TABELA BASE DO EXEL , COM OS DADOS QUE FORAM BUSCADOS, E UMA CONSULTA NO BANCO DE DADOS

In [ ]:
df = pd.read_excel("Base_dados.xlsx", usecols=['EVADIDO','DP','INADIMPLENTE','QTD_MESES_INADI','CURSO','QTD_DP','MEDIA_DP_POR_INADI','PERFIL_RISCO','MEDIA_COBRANCAS','QTD_PERIODOS'], engine = "openpyxl")
df = df.fillna('n') # Preencher valores null com 'n'
print(df.columns)

### TRANSFORMA AS COLUNAS DE SIM E NÃO, EM BINÁRIAS, E AS DEMAIS COLUNAS EM FLOAT, TRANSFORMANDO OS VALORES NULOS PARA ZERO

In [ ]:
colunas = ['EVADIDO','DP','INADIMPLENTE','QTD_MESES_INADI','CURSO','QTD_DP','MEDIA_DP_POR_INADI','PERFIL_RISCO','MEDIA_COBRANCAS','QTD_PERIODOS']


df[colunas] = df[colunas].fillna("SEM VALOR").astype(str).apply(lambda x: x.str.strip().str.upper())

# Convertendo binários de 'S' para 1 e outros para 0
df['DP'] = df['DP'].apply(lambda valor: 1 if valor == 'S' else 0)
df['EVADIDO'] = df['EVADIDO'].apply(lambda valor: 1 if valor == 'S' else 0)
df['INADIMPLENTE'] = df['INADIMPLENTE'].apply(lambda valor: 1 if valor == 'S' else 0)


# Convertendo colunas numéricas de volta ao tipo correto
df['QTD_MESES_INADI'] = pd.to_numeric(df['QTD_MESES_INADI'], errors='coerce').fillna(0).astype(int)
df['QTD_DP'] = pd.to_numeric(df['QTD_DP'], errors='coerce').fillna(0).astype(int)
df['MEDIA_DP_POR_INADI'] = pd.to_numeric(df['MEDIA_DP_POR_INADI'], errors='coerce').fillna(0).astype(float)
df['MEDIA_COBRANCAS'] = pd.to_numeric(df['MEDIA_COBRANCAS'], errors='coerce').fillna(0).astype(float)
df['PERFIL_RISCO'] = pd.to_numeric(df['PERFIL_RISCO'], errors='coerce').fillna(0).astype(float)
df['QTD_PERIODOS'] = pd.to_numeric(df['QTD_PERIODOS'], errors='coerce').fillna(0).astype(float)
df['CURSO'] = pd.to_numeric(df['CURSO'], errors='coerce').fillna(0).astype(float)


print(df)
print(df[colunas].dtypes)

### DECLARA AS COLUNAS DE BASE(X), E A DE SAÍDA (Y), OS EVADIDOS

In [ ]:
X = df[['CURSO', 'INADIMPLENTE', 'DP', 'QTD_DP', 'QTD_MESES_INADI', 'MEDIA_DP_POR_INADI','MEDIA_COBRANCAS','PERFIL_RISCO','QTD_PERIODOS']]
y = df['EVADIDO']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)

### FAZ UM <B>PIPELINE</B>, AONDE ELE APLICA O BALANCEAMENTO COM O <B>ADASYN</B>, E O MODELO DA MACHINE , O <B>RandomForestclassifier</B>

In [ ]:

pipeline = Pipeline([
    ('adasyn', ADASYN(random_state=42,sampling_strategy=0.7)),
    ('rf', RandomForestClassifier(random_state=42))
])

### UTILIZA O <B>GridSearchCV</B> PARA FAZER UM SEGUNDO BALANCEAMENTO, GERANDO DADOS SINTÉTICOS, PARA TREINAR A MACHINE

In [ ]:


param_grid = {
    'rf__n_estimators': [100, 200, 500],
    'rf__max_depth': [10, 20, None],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['sqrt', 'log2']
}


grid = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5, n_jobs=-1)
grid.fit(X, y)  


best_model_ead = grid.best_estimator_

### AJUSTE NO THRESHOLD PARA TENDENCIONAR A MACHINE NO VALOR 1 (EVADIDOS)

In [ ]:
y_proba = grid.predict_proba(X)[:, 1]
y_pred = (y_proba > 0.6).astype(int)  #


### TESTES PARA VER O MELHOR THRESHOLD DE DECISÃO

In [ ]:
y_pred_all = y_proba.copy()  
pr_table = []
print("\n### Resultados CatBoost com melhores params ###\n")

# Loop para testar vários thresholds de decisão
for thresh in [0.44,0.442,0.443,0.444,0.445,0.446,0.447,0.448,0.449,0.45]:
  
    y_pred = (y_pred_all > thresh).astype(int)
    print(f"--- Threshold: {thresh} ---")
    print(classification_report(y, y_pred))
    pr_table.append({
        'threshold': np.append(thresh, 1),
        'precision': float(classification_report(y, y_pred).split('\n')[3].split()[1]),
        'recall': float(classification_report(y, y_pred).split('\n')[3].split()[2])
    })


pr_table = pd.DataFrame(pr_table)

relevantes = pr_table[(pr_table['recall'] < 0.6) & (pr_table['precision'] > 0.5)]

relevantes = relevantes.sort_values(by='recall', ascending=False)

print(relevantes.head(10))

## MATRIZ DE CONFUSÃO PARA VERMOS OS RESULTADOS

In [ ]:


y_pred = (y_pred_all > 0.447).astype(int)
matriz = confusion_matrix(y, y_pred) #y (valores reais) e y_pred (valores previstos)
sns.heatmap(matriz, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Não Evadiu (previsão)', 'Evadiu (previsão)'],
            yticklabels=['Não Evadiu (real)', 'Evadiu (real)'])
plt.xlabel('Previsão')
plt.ylabel('Realidade')
plt.title('Matriz de Confusão(0.447),pipeline')
plt.show()

## IMPORTANDO NOSSO MODELO PARA APLICAR DIA A DIA

In [ ]:
import pickle

# Defina seu threshold
threshold = 0.447

# Empacote o modelo e o threshold em um dicionário
modelo_completo = {
    'modelo': best_model_ead,
    'threshold': threshold
}

# Salve 
with open('modelo_lgb_com_threshold_0.447_real.pkl', 'wb') as f:
    pickle.dump(modelo_completo, f)
